# Expression API

式を動的に構築し、インスタンス化するAPI

In [1]:
import duckdb

flights = duckdb.sql("""
SELECT
    *
FROM 'https://github.com/plotly/datasets/raw/refs/heads/master/2015_flights.parquet'
""").limit(1000)
flights.show()

┌─────────────────┬───────────────┬──────────┬─────────────────────┐
│ DEPARTURE_DELAY │ ARRIVAL_DELAY │ DISTANCE │ SCHEDULED_DEPARTURE │
│     double      │    double     │  int64   │       double        │
├─────────────────┼───────────────┼──────────┼─────────────────────┤
│           -11.0 │         -22.0 │     1448 │ 0.08333333333333333 │
│            -8.0 │          -9.0 │     2330 │ 0.16666666666666666 │
│            -2.0 │           5.0 │     2296 │  0.3333333333333333 │
│            -5.0 │          -9.0 │     2342 │  0.3333333333333333 │
│            -1.0 │         -21.0 │     1448 │  0.4166666666666667 │
│            -5.0 │           8.0 │     1589 │  0.4166666666666667 │
│            -6.0 │         -17.0 │     1299 │  0.4166666666666667 │
│            14.0 │         -10.0 │     2125 │                 0.5 │
│           -11.0 │         -13.0 │     1464 │                 0.5 │
│             3.0 │         -15.0 │     1747 │                 0.5 │
│              ·  │            ·  

## Column Expression

列名のインスタンス化

In [2]:
flights.select(duckdb.ColumnExpression("DEPARTURE_DELAY"))

┌─────────────────┐
│ DEPARTURE_DELAY │
│     double      │
├─────────────────┤
│           -11.0 │
│            -8.0 │
│            -2.0 │
│            -5.0 │
│            -1.0 │
│            -5.0 │
│            -6.0 │
│            14.0 │
│           -11.0 │
│             3.0 │
│              ·  │
│              ·  │
│              ·  │
│            NULL │
│            -6.0 │
│            -6.0 │
│             0.0 │
│             4.0 │
│            29.0 │
│            -2.0 │
│            -7.0 │
│            -7.0 │
│            -8.0 │
├─────────────────┤
│    1000 rows    │
│   (20 shown)    │
└─────────────────┘

- 複数列の場合はリストに `ColumnExpression` インスタンスを入れる
- `ColumnExpression` インスタンスは式を評価、メソッドの実行ができる

```{seealso}
Common Operations

https://duckdb.org/docs/api/python/expression#common-operations
```

In [3]:
cols = [
    duckdb.ColumnExpression("DEPARTURE_DELAY") - 1,
    duckdb.ColumnExpression("ARRIVAL_DELAY").isnull(),
]
flights.select(*cols)

┌───────────────────────┬─────────────────────────┐
│ (DEPARTURE_DELAY - 1) │ (ARRIVAL_DELAY IS NULL) │
│        double         │         boolean         │
├───────────────────────┼─────────────────────────┤
│                 -12.0 │ false                   │
│                  -9.0 │ false                   │
│                  -3.0 │ false                   │
│                  -6.0 │ false                   │
│                  -2.0 │ false                   │
│                  -6.0 │ false                   │
│                  -7.0 │ false                   │
│                  13.0 │ false                   │
│                 -12.0 │ false                   │
│                   2.0 │ false                   │
│                    ·  │  ·                      │
│                    ·  │  ·                      │
│                    ·  │  ·                      │
│                  NULL │ true                    │
│                  -7.0 │ false                   │
│           

## Star Expression

- すべての列を選択
- オプション（ `exclude` ）で特定の列を除外できる

In [4]:
flights.select(duckdb.StarExpression(exclude=["ARRIVAL_DELAY", "SCHEDULED_DEPARTURE"]))

┌─────────────────┬──────────┐
│ DEPARTURE_DELAY │ DISTANCE │
│     double      │  int64   │
├─────────────────┼──────────┤
│           -11.0 │     1448 │
│            -8.0 │     2330 │
│            -2.0 │     2296 │
│            -5.0 │     2342 │
│            -1.0 │     1448 │
│            -5.0 │     1589 │
│            -6.0 │     1299 │
│            14.0 │     2125 │
│           -11.0 │     1464 │
│             3.0 │     1747 │
│              ·  │       ·  │
│              ·  │       ·  │
│              ·  │       ·  │
│            NULL │      546 │
│            -6.0 │      925 │
│            -6.0 │     2039 │
│             0.0 │      990 │
│             4.0 │     1642 │
│            29.0 │      496 │
│            -2.0 │      842 │
│            -7.0 │      639 │
│            -7.0 │     2434 │
│            -8.0 │      130 │
├─────────────────┴──────────┤
│    1000 rows (20 shown)    │
└────────────────────────────┘

## Constant Expression

- 定数
- Case ExpressionやFunction Expressionで利用できる

In [5]:
flights.select(duckdb.ConstantExpression(1))

┌────────────┐
│     1      │
│   int32    │
├────────────┤
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          · │
│          · │
│          · │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
│          1 │
├────────────┤
│ 1000 rows  │
│ (20 shown) │
└────────────┘

## Case Expression

- `CASE WHEN (...) THEN (...) ELSE (...) END` 式を評価
- デフォルトでは `ELSE` は `NULL` となり、 `.else(value = ...)` で `ELSE` を設定
- `WHEN (...) THEN (...)` ブロックを `.when(condition = ..., value = ...)` として追加できる

```{seealso}
CASE Statement

https://duckdb.org/docs/sql/expressions/case
```

In [6]:
distance = duckdb.ColumnExpression("DISTANCE")
case = duckdb.CaseExpression(
    condition=(distance < 1000),
    value=duckdb.ConstantExpression(0),
).otherwise(distance)
flights.select(case)

┌───────────────────────────────────────────────────────────┐
│ CASE  WHEN ((DISTANCE < 1000)) THEN (0) ELSE DISTANCE END │
│                           int64                           │
├───────────────────────────────────────────────────────────┤
│                                                      1448 │
│                                                      2330 │
│                                                      2296 │
│                                                      2342 │
│                                                      1448 │
│                                                      1589 │
│                                                      1299 │
│                                                      2125 │
│                                                      1464 │
│                                                      1747 │
│                                                         · │
│                                                         · │
│       

## Function Expression

- 関数を式として記述
- 任意の数の引数を指定できる

In [7]:
flights.select(
    duckdb.FunctionExpression("abs", duckdb.ColumnExpression("DEPARTURE_DELAY"))
)

┌──────────────────────┐
│ abs(DEPARTURE_DELAY) │
│        double        │
├──────────────────────┤
│                 11.0 │
│                  8.0 │
│                  2.0 │
│                  5.0 │
│                  1.0 │
│                  5.0 │
│                  6.0 │
│                 14.0 │
│                 11.0 │
│                  3.0 │
│                   ·  │
│                   ·  │
│                   ·  │
│                 NULL │
│                  6.0 │
│                  6.0 │
│                  0.0 │
│                  4.0 │
│                 29.0 │
│                  2.0 │
│                  7.0 │
│                  7.0 │
│                  8.0 │
├──────────────────────┤
│ 1000 rows (20 shown) │
└──────────────────────┘